# Multi-Type-TD-TSR Demo Notebook
## 1. Install all the prerequisite


环境变量我已经配好了，直接跳转到带边框的表格识别的部分即可。

In [1]:
# install CUDA 10.1 if necessary
# !apt install cuda-10-1 -y
!cat /usr/local/cuda/version.txt

CUDA Version 10.1.243


In [ ]:
# install dependencies: 
!pip install pyyaml==5.1
!pip install torch==1.7
!pip install torchvision==0.8.1
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab

# install detectron2: (Colab has CUDA 10.1 + torch 1.7)
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions

!apt install tesseract-ocr 
!pip install pytesseract

import torch, torchvision
torch.__version__
assert torch.__version__.startswith("1.7")
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.7/index.html
# exit(0)  # After installation, you need to "restart runtime" in Colab. This line can also restart runtime

## 以下是安装PaddlePaddle OCR的步骤
[旧版本](https://www.paddlepaddle.org.cn/install/old?docurl=/documentation/docs/zh/install/conda/windows-conda.html)

In [ ]:
!python -m pip install paddlepaddle-gpu==2.3.2.post101 -f https://www.paddlepaddle.org.cn/whl/linux/mkl/avx/stable.html

## 2. Clone the Repository 

In [ ]:
!export http_proxy=http://192.168.1.174:12798 && export https_proxy=http://192.168.1.174:12798 # autodl学术加速
!git clone https://github.com/Psarpei/Multi-Type-TD-TSR.git
!mv Multi-Type-TD-TSR/ Multi_Type_TD_TSR

## 3. Download Table Detection Weights and Configuration File

In [ ]:
# colab预装gdown，zg不能访问google drive，手动上传文件
# 下载的文件为 model_final.pth All_X152.yaml Base-RCNN-FPN.yaml
!gdown --id 11FgFTy0MyVUMGd00T_InEDaarB4qAlP8
!gdown --id 1WBk6kHHyvyEzoPBsRr2BvFY51zURjd4R
!gdown --id 1PfA2uws919gc893-x9uMIz06zWEko8nF 

## 4. Import Everything

In [5]:
# install open-cv2
!conda install -c conda-forge opencv

Solving environment: done

# All requested packages already installed.



In [8]:
!pip install pandas

Looking in indexes: https://repo.huaweicloud.com/repository/pypi/simple
     |████████████████████████████████| 12.2 MB 23.9 MB/s eta 0:00:01


In [3]:
import detectron2
import Multi_Type_TD_TSR.google_colab.deskew as deskew
import Multi_Type_TD_TSR.google_colab.table_detection as table_detection
import Multi_Type_TD_TSR.google_colab.table_structure_recognition_all as tsra
import Multi_Type_TD_TSR.google_colab.table_structure_recognition_lines as tsrl
import Multi_Type_TD_TSR.google_colab.table_structure_recognition_wol as tsrwol
import Multi_Type_TD_TSR.google_colab.table_structure_recognition_lines_wol as tsrlwol
import Multi_Type_TD_TSR.google_colab.table_xml as txml
import Multi_Type_TD_TSR.google_colab.table_ocr as tocr
import pandas as pd
import os
import json
import itertools
import random
from detectron2.utils.logger import setup_logger
# import some common libraries
import numpy as np
import cv2
import matplotlib.pyplot as plt
# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from detectron2.data import DatasetCatalog, MetadataCatalog
from google.colab.patches import cv2_imshow
setup_logger()

ModuleNotFoundError: No module named 'google.colab'

## 5. Initialize Table Detection Detector

In [1]:
#create detectron config
cfg = get_cfg()

#set yaml
cfg.merge_from_file('./All_X152.yaml')

#set model weights
cfg.MODEL.WEIGHTS = './model_final.pth' # Set path model .pth

predictor = DefaultPredictor(cfg) 

NameError: name 'get_cfg' is not defined

#Table Rotation

In [2]:
# path to the image scan of the document
file = "./Multi_Type_TD_TSR/images/rotated_example.jpeg" 

# load the image from disk
original_image = cv2.imread(file)
deskewed_image = deskew.deskewImage(original_image)

print("ORIGINAL IMAGE:")
cv2_imshow(original_image)

print()

print("DESKEWED IMAGE:")
cv2_imshow(deskewed_image)

NameError: name 'cv2' is not defined

# Table Detection

In [ ]:
document_img = cv2.imread("/content/Multi_Type_TD_TSR/images/color_invariance_example.jpg")
table_detection.plot_prediction(document_img, predictor)


In [ ]:
table_list, table_coords = table_detection.make_prediction(document_img, predictor)

#Table Structure Recognition Partially Bordered Tables
# with Color Invariance Pre-Processing

In [ ]:
list_table_boxes = []

for table in table_list:
    finalboxes, output_img = tsra.recognize_structure(table)
    list_table_boxes.append(finalboxes)

# XML Output 

In [ ]:
txml.output_to_xml(table_coords, list_table_boxes)

#Table Structure Recognition Bordered Tables

In [ ]:
bordered_table = cv2.imread("/content/test1.png")
list_table_boxes = []

table_list = [bordered_table]

for table in table_list:
    finalboxes, output_img = tsrl.recognize_structure(table)
    list_table_boxes.append(finalboxes)

# Table Structure Recognition Unbordered Tables

In [ ]:
unbordered_table = cv2.imread("/content/Multi_Type_TD_TSR/images/unbordered_example.jpg")
list_table_boxes = []

table_list = [unbordered_table]

for table in table_list:
    finalboxes, output_img = tsrwol.recognize_structure(table)
    list_table_boxes.append(finalboxes)

#Table Structure Recognition Partially Bordered Tables

In [ ]:
document_example = cv2.imread("/content/Multi_Type_TD_TSR/images/example.jpg")

table_listocrt, table_coords = table_detection.make_prediction(document_example, predictor)
list_table_boxes = []
for table in table_list:
    finalboxes, output_img = tsrlwol.recognize_structure(table)
    list_table_boxes.append(finalboxes)

#Additional Feature: OCR with PyTesserect (Output in CSV)

In [ ]:
tocr.output_to_csv(list_table_boxes[0], output_img)